<a href="https://colab.research.google.com/github/phanlinh0803/atom-assignments/blob/Phan-Linh-assigntment-03/ATOM_%7C_Phan_Linh_assignment_03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

0/ Load Modules

In [64]:
import requests #-> Để gọi API
import re #-> Để xử lý data dạng string
from datetime import datetime as dt #-> Để xử lý data dạng datetime
import gspread #-> Để update data lên Google Spreadsheet
from gspread_dataframe import set_with_dataframe #-> Để update data lên Google Spreadsheet
import pandas as pd #-> Để update data dạng bản
import json 
from oauth2client.service_account import ServiceAccountCredentials #-> Để nhập Google Spreadsheet Credentials
import os

1/ Slack API: User List

In [65]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving env_variable.json to env_variable (1).json
User uploaded file "env_variable.json" with length 232 bytes


In [69]:
!ls

'env_variable (1).json'   rugged-destiny-271710-68c75a121c99.json
 env_variable.json	  sample_data


In [70]:
with open('env_variable (1).json', 'r') as j:
    json_data = json.load(j)
    # print(json_data)

In [71]:
## Load SLACK_BEARER_TOKEN
os.environ['SLACK_BEARER_TOKEN'] = json_data['SLACK_BEARER_TOKEN']

In [72]:
## Gọi API từ Endpoints (Input - Token được đưa vào Headers)
## Challenge: Thử gọi API này bằng Postman
endpoint = "https://slack.com/api/users.list"
headers = {"Authorization": "Bearer {}".format(os.environ['SLACK_BEARER_TOKEN'])}
response_json = requests.post(endpoint, headers=headers).json() 
user_dat = response_json['members']

TODO #1

In [73]:
## Loop qua JSON file và extract các thông tin quan trọng (id, name, display_name, real_name_normalized, title, phone, is_bot)
## Hint: Bạn có thể dùng Postman hoặc in user_dat JSON để xem cấu trúc (schema), dùng Ctrl+F để tìm các keys (id, name, display_name, real_name_normalized, title, phone, is_bot)
user_dict = {'user_id':[], 'name':[], 'display_name':[],'real_name':[],'title':[],'phone':[],'is_bot':[]}
for i in range(len(user_dat)):
  user_dict['user_id'].append(user_dat[i]['id'])
  user_dict['name'].append(user_dat[i]['name'])
  user_dict['display_name'].append(user_dat[i]['profile']['display_name'])
  user_dict['real_name'].append(user_dat[i]['profile']['real_name_normalized'])
  user_dict['title'].append(user_dat[i]['profile']['title'])
  user_dict['phone'].append(user_dat[i]['profile']['phone'])
  user_dict['is_bot'].append(user_dat[i]['is_bot'])

In [74]:
user_df = pd.DataFrame(user_dict) ## Dùng pandas để convert dictionaries thành bảng
user_df.head(5) ## Chỉ in 5 dòng đầu (chủ yếu để xem cấu trúc)

,user_id,name,display_name,real_name,title,phone,is_bot
0,USLACKBOT,slackbot,Slackbot,Slackbot,,,False
1,U01AT4T75JB,loclexuan26392,Loc Le Xuan,Loc Le Xuan,,,False
2,U01AVDY7JET,locle.ds,Loc Le Xuan,Loc Le Xuan,,,False
3,U01BE2PR6LU,maianhdang.ftu,MAD,Dặng Huỳnh Mai Anh,Technical Contents,,False
4,U01C48T7S1J,huyenhoang.design,Thanh Huyen Hoang,Thanh Huyen Hoang,,,False


In [75]:
user_df[user_df.display_name == 'MAD'] ## Lọc thông tin của MAD, trên DataFrame

,user_id,name,display_name,real_name,title,phone,is_bot
3,U01BE2PR6LU,maianhdang.ftu,MAD,Dặng Huỳnh Mai Anh,Technical Contents,,False


TODO#2

In [76]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving rugged-destiny-271710-68c75a121c99.json to rugged-destiny-271710-68c75a121c99 (1).json
User uploaded file "rugged-destiny-271710-68c75a121c99.json" with length 2342 bytes


In [77]:

!ls

'env_variable (1).json'
 env_variable.json
'rugged-destiny-271710-68c75a121c99 (1).json'
 rugged-destiny-271710-68c75a121c99.json
 sample_data


In [78]:
## Authorize bằng JSON
scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name(
    'rugged-destiny-271710-68c75a121c99 (1).json', scope)
gc = gspread.authorize(credentials)
print("DONE!")

DONE!


In [79]:
## Gọi SLACK API để list tất cả các channel
endpoint = "https://slack.com/api/conversations.list"
headers = {"Authorization": "Bearer {}".format(os.environ['SLACK_BEARER_TOKEN'])}
response = requests.post(endpoint, headers=headers).json() 
channel_ls = response['channels']

In [80]:
channel_ls[0] ## Thử extract record đầu tiên để xem schema  => name: general, id: C01B4PVGLVB

{'created': 1600856703,
 'creator': 'U01BE2PR6LU',
 'id': 'C01B4PVGLVB',
 'is_archived': False,
 'is_channel': True,
 'is_ext_shared': False,
 'is_general': True,
 'is_group': False,
 'is_im': False,
 'is_member': True,
 'is_mpim': False,
 'is_org_shared': False,
 'is_pending_ext_shared': False,
 'is_private': False,
 'is_shared': False,
 'name': 'general',
 'name_normalized': 'general',
 'num_members': 65,
 'parent_conversation': None,
 'pending_connected_team_ids': [],
 'pending_shared': [],
 'previous_names': [],
 'purpose': {'creator': 'U01BE2PR6LU',
  'last_set': 1600856703,
  'value': 'This is the one channel that will always include everyone. It’s a great spot for announcements and team-wide conversations.'},
 'shared_team_ids': ['T01B7SGGMLJ'],
 'topic': {'creator': '', 'last_set': 0, 'value': ''},
 'unlinked': 0}

Option 1: Update data => Google SpreadSheet

In [81]:
# ACCES GOOGLE SHEET
sheet_index_no = 0
spreadsheet_key = '1DK_TW9a1yBOmGLzOiR7WcbofkUJrk6qw2SSp84625pE' # input SPREADSHEET_KEY HERE
sh = gc.open_by_key(spreadsheet_key)
worksheet = sh.get_worksheet(sheet_index_no) #-> 0 - first sheet, 1 - second sheet etc. 

# APPEND DATA TO SHEET
set_with_dataframe(worksheet, user_df) #-> Upload user_df vào Sheet đầu tiên trong Spreadsheet

Option 2: Ai đã nộp bài?

In [85]:
## Gọi SLACK API để list tất cả các channel
endpoint = "https://slack.com/api/conversations.list"
headers = {"Authorization": "Bearer {}".format(os.environ['SLACK_BEARER_TOKEN'])}
response = requests.post(endpoint, headers=headers).json() 
channel_ls = response['channels']

In [86]:
channel_ls[0] ## Thử extract record đầu tiên để xem schema  => name: general, id: C01B4PVGLVB

{'created': 1600856703,
 'creator': 'U01BE2PR6LU',
 'id': 'C01B4PVGLVB',
 'is_archived': False,
 'is_channel': True,
 'is_ext_shared': False,
 'is_general': True,
 'is_group': False,
 'is_im': False,
 'is_member': True,
 'is_mpim': False,
 'is_org_shared': False,
 'is_pending_ext_shared': False,
 'is_private': False,
 'is_shared': False,
 'name': 'general',
 'name_normalized': 'general',
 'num_members': 65,
 'parent_conversation': None,
 'pending_connected_team_ids': [],
 'pending_shared': [],
 'previous_names': [],
 'purpose': {'creator': 'U01BE2PR6LU',
  'last_set': 1600856703,
  'value': 'This is the one channel that will always include everyone. It’s a great spot for announcements and team-wide conversations.'},
 'shared_team_ids': ['T01B7SGGMLJ'],
 'topic': {'creator': '', 'last_set': 0, 'value': ''},
 'unlinked': 0}

TODO#3

Slack API: List messages trong 1 channel

In [82]:
endpoint = "https://slack.com/api/conversations.history"
data = {"channel": "C01U6P7LZ8F"} ## This is ID of assignment#1 channel
headers = {"Authorization": "Bearer {}".format(os.environ['SLACK_BEARER_TOKEN'])}

In [83]:
## Load SLACK_BEARER_TOKEN
os.environ['SLACK_BEARER_TOKEN'] = json_data['SLACK_BEARER_TOKEN']

In [84]:
response_json = requests.post(endpoint, data=data, headers=headers).json()
msg_ls = response_json['messages']

In [104]:
msg_ls[33]

{'attachments': [{'fallback': 'Google Colaboratory',
   'from_url': 'https://colab.research.google.com/drive/1p8DoHN5_J9IAgwfgeMgpoU-s7TQTWdd9?authuser=1#scrollTo=qgcHbg2Xcq3x',
   'id': 1,
   'image_bytes': 4534,
   'image_height': 250,
   'image_url': 'https://colab.research.google.com/img/colab_favicon_256px.png',
   'image_width': 250,
   'original_url': 'https://colab.research.google.com/drive/1p8DoHN5_J9IAgwfgeMgpoU-s7TQTWdd9?authuser=1#scrollTo=qgcHbg2Xcq3x',
   'service_icon': 'https://colab.research.google.com/img/favicon.ico?vrz=colab-20210421-060109-RC00_369621175',
   'service_name': 'colab.research.google.com',
   'title': 'Google Colaboratory',
   'title_link': 'https://colab.research.google.com/drive/1p8DoHN5_J9IAgwfgeMgpoU-s7TQTWdd9?authuser=1#scrollTo=qgcHbg2Xcq3x'}],
 'blocks': [{'block_id': 'jlY3',
   'elements': [{'elements': [{'text': 'Em gửi bài nộp Assignment 1 dưới đây ạ. Chúc mọi người buổi tối vui vẻ!\nTask1: ',
       'type': 'text'},
      {'type': 'link',
 

In [88]:
not_learners_id = ['U01BE2PR6LU']

In [103]:
## Summarize all submitted assignments + reviews cnt
not_learners_id = ['U01BE2PR6LU'] # -> Remove MA from the user_id
github, reply_count, reply_users_count, reply_users, latest_reply = '','','','',''
for i in range(50):
  ts = dt.fromtimestamp(float(msg_ls[i]['ts'])) # -> Convert timestamp Epoch thành dàng dễ đọc
  user = msg_ls[i]['user'] # -> Lấy thông tin người post messages
  if msg_ls[i]['user'] not in not_learners_id:
    if 'attachments' in msg_ls[i].keys():
      #print(msg_ls[i].keys())
      text = msg_ls[i]['text']
      github_link = re.findall('(?:https?://)?(?:www[.])?github[.]com/[\w-]+/?', text) #-> Submission là các message có link github
      #print(msg_ls[i])
      if len(github_link) > 0: github = github_link[0]
      if 'reply_count' in msg_ls[i].keys(): reply_count = msg_ls[i]['reply_count'] #-> Extract số review
      if 'reply_users_count' in msg_ls[i].keys(): reply_users_count = msg_ls[i]['reply_users_count']
      if 'reply_users' in msg_ls[i].keys(): reply_users = msg_ls[i]['reply_users']
      if 'latest_reply' in msg_ls[i].keys(): latest_reply = dt.fromtimestamp(float(msg_ls[i]['latest_reply']))
      print(ts, user, reply_users_count, reply_users, latest_reply, github)


2021-05-10 04:51:46.004700 U01UJ9LG5U5 2 ['U01USGKQ771', 'U01V00KHHHP'] 2021-05-15 09:14:31.003600 https://github.com/danhpcv/
2021-05-10 04:35:39.003800 U01UMC08KL2 2 ['U01UTGS5ZNY', 'U01UMC08KL2'] 2021-05-18 08:09:59.000800 https://github.com/auslynnguyen/
2021-05-08 06:31:59.005200 U01UMJ36QD9 1 ['U01UMBX2X18'] 2021-05-08 06:36:43.006000 https://github.com/auslynnguyen/
2021-05-04 13:55:46.001800 U01UTGVPE7N 1 ['U01UMBX2X18'] 2021-05-08 06:36:43.006000 https://github.com/hoaintp/
2021-05-03 17:41:18.002400 U01U6JQB695 2 ['U01UMC0N5U2', 'U01U6JQB695'] 2021-05-11 03:10:06.001300 https://github.com/Tenderriver/
2021-05-03 08:51:19.001200 U01V00KHHHP 1 ['U01UMBZHU2W'] 2021-05-14 17:51:26.024600 https://github.com/saturn1101/
2021-04-26 06:53:56.000800 U01UMBZHU2W 2 ['U01UMJ451V1', 'U01UMBZHU2W'] 2021-05-14 17:52:35.025100 https://github.com/saturn1101/
2021-04-24 02:01:27.043200 U01VB63LRNC 1 ['U01UMJ451V1'] 2021-05-11 12:37:14.003100 https://github.com/AnhThuNguyenHuynh/
2021-04-24 01:

TODO#4

In [141]:
# ACCES GOOGLE SHEET
sheet_index_no = 0
spreadsheet_key = '1ynFdH07EM0U62jVj6NstLnBZjhipEgvWxd4_oDe3AW4' # input SPREADSHEET_KEY HERE
sh = gc.open_by_key(spreadsheet_key)
worksheet = sh.get_worksheet(sheet_index_no) #-> 0 - first sheet, 1 - second sheet etc. 

# APPEND DATA TO SHEET
y, x = dataframe.shape
index_col_size = 0
column_header_size = 0
set_with_dataframe(worksheet,msg_ls) #-> Upload msg_ls vào Sheet đầu tiên trong Spreadsheet

APIError: ignored